In [1]:
import torch

print(f"Versión de Torch: {torch.__version__}")
print(f"¿Soporta BFloat16 en CPU?: {torch.cuda.is_bf16_supported() if torch.cuda.is_available() else 'Probando en CPU...'}")

# Test rápido de tensor en bfloat16
try:
    x = torch.tensor([1.0, 2.0], dtype=torch.bfloat16)
    print("¡Perfecto! Tu sistema maneja tensores bfloat16 sin problemas.")
except:
    print("Tu CPU actual podría tener problemas con bfloat16. Usaremos float32 si es necesario.")

Versión de Torch: 2.10.0+cpu
¿Soporta BFloat16 en CPU?: Probando en CPU...
¡Perfecto! Tu sistema maneja tensores bfloat16 sin problemas.


In [2]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, TextIteratorStreamer
from PIL import Image
import requests
import time
from io import BytesIO
from threading import Thread

In [3]:
torch.set_num_threads(8)

In [4]:
model_id = "google/medgemma-1.5-4b-it"

# Cargamos en CPU explícitamente
device = "cpu"

print("Cargando modelo en RAM... (Esto puede tardar un poco)")
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # O torch.float32 si tu CPU es antigua
    device_map={"": device},    # Forzamos todo a CPU
    low_cpu_mem_usage=True      # Optimiza el uso de RAM durante la carga
)


Cargando modelo en RAM... (Esto puede tardar un poco)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:


url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"

# Añadimos un Header para que el servidor no rechace la petición
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers, stream=True)
    response.raise_for_status() # Verifica si la descarga fue exitosa (Error 404, 403, etc.)
    
    # Abrimos la imagen directamente desde el contenido binario
    image = Image.open(BytesIO(response.content)).convert("RGB")
    print("Imagen cargada con éxito.")
    
except requests.exceptions.RequestException as e:
    print(f"Error al descargar la imagen: {e}")
except Exception as e:
    print(f"Error al identificar la imagen: {e}")


Imagen cargada con éxito.


In [8]:

messages = [
    {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": "Analyze this X-ray."}]}
]

prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

streamer = TextIteratorStreamer(processor, skip_special_tokens=True, skip_prompt=True)
generation_kwargs = dict(
    **inputs,
    streamer=streamer,
    max_new_tokens=1024,
    eos_token_id=processor.tokenizer.eos_token_id,
    do_sample=False,
    temperature=0.0
)

thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

print("Generando respuesta en CPU...")
start_time = time.time()

# with torch.no_grad():
#     output_ids = model.generate(**inputs, max_new_tokens=100)

# response = processor.decode(output_ids[0], skip_special_tokens=True)

# 5. Imprimir el texto según llega
print("\n--- Respuesta de MedGemma (Generando en tiempo real) ---\n")
for new_text in streamer:
    print(new_text, end="", flush=True)

thread.join()
print("\n\n--- Generación completada ---")

end_time = time.time()

print(f"\n--- Respuesta ---\n{response}")
print(f"\nTiempo tardado: {end_time - start_time:.2f} segundos.")

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Generando respuesta en CPU...

--- Respuesta de MedGemma (Generando en tiempo real) ---

Based on the chest X-ray provided, here's a general analysis:

**Overall Impression:**

The image shows a normal chest X-ray. The heart size appears within normal limits, the lungs are clear, and there are no obvious signs of consolidation, pleural effusion, or pneumothorax. The mediastinum is unremarkable.

**Specific Observations:**

*   **Heart Size:** The heart size appears to be within normal limits.
*   **Lung Fields:** The lung fields are clear bilaterally, with no evidence of consolidation, masses, or significant infiltrates.
*   **Mediastinum:** The mediastinum appears normal in width and alignment.
*   **Bones:** The ribs, clavicles, and other bony structures are intact.
*   **Diaphragm:** The diaphragms are well-defined.

**Possible Considerations (depending on clinical context):**

*   **Age:** The patient's age is not provided, but the image appears to be of a younger individual.
*   *

KeyboardInterrupt: 